In [634]:
import pandas as pd

In [635]:
# Variablen

# Mandatory
tm_result = "tm_result"
df = pd.read_csv("tablematch.csv", sep = ";", decimal = ",") # TODO Umbenennen der Variable
df_import = pd.read_csv("test_data.csv", sep = ";", decimal = ",") # TODO Umbenennen der Variable

# Optional
tm_active = "tm_active"
tm_order = "tm_order"
tm_category = "tm_category"
input_category = None
tm_comment = "tm_comment"

case_sensitive = True # Nutzt Upper in Funktionen

inplace = False # TODO Funktion einfügen

In [636]:
# Nur aktive Regeln behalten
if tm_active is not None:
    df = df.loc[df[tm_active] == True,:].copy()
    df.drop(columns = [tm_active], inplace = True)

# NAch Category Filtern
if input_category is not None and tm_category is not None:
    df = df.loc[df[tm_category] == input_category,:].copy()
df.drop(columns = [tm_category], inplace = True)

# Sortieren nach tm_id
if tm_order is not None:
    df.sort_values(by = tm_order, inplace = True)
    df.drop(columns = [tm_order], inplace = True)

# Drop Comment
if tm_comment is not None:
    df.drop(columns = [tm_comment], inplace = True)

# Alle wichtigen Spalten sammeln
columns_all = df.columns.to_list()
columns_all.remove(tm_result)

# Dict fürs Mapping von Boolean
booleanDictionary_Upper = {True: 'True', False: 'False'}

# Jede Spalte durchgehen und bei Bedarf verändern
for column in columns_all:
    # Ersetzt Boolean Werte durch Strings
    df[column] = df[column].replace(booleanDictionary_Upper) # Muss ausgeführt werden, da sonst str.upper() einen Fehler wirft.

    # Ändert Datentyp in objects
    if df[column].dtype != object:
        df[column] = df[column].astype(object)
        df[column] = df[column].map(str) # Sonst werden Zahlen nicht richtig erkannt
    
    # Füllt Null mit leeren Strings auf
    df[column].fillna('', inplace = True) # NA werden zu leeren Strings

    # Macht die Strings groß
    if not case_sensitive:
        df[column] = df[column].str.upper()

    # Teilt die Werte bei einem Komma auf
    df[column] = df[column].str.split(pat = ",")
    df = df.explode(column)
    df[column] = df[column].str.strip()

    # Reset Index
    df.reset_index(drop = True, inplace = True)

    # Aufbereitung der Regeln
    for index, row in df.iterrows():
        if row[column] == '':
            df.loc[index,column] = None
        elif row[column][0:2] == '>=':
            df.loc[index,column] = 'df_input["' + column + '"] >= ' + row[column][2:]
        elif row[column][0:2] == '<=':
            df.loc[index,column] = 'df_input["' + column + '"] <= ' + row[column][2:]
        elif row[column][0] == '>':
            df.loc[index,column] = 'df_input["' + column + '"] > ' + row[column][1:]
        elif row[column][0] == '<':
            df.loc[index,column] = 'df_input["' + column + '"] < ' + row[column][1:]
        elif row[column][0:2] == '==':
            df.loc[index,column] = 'df_input["' + column + '"] == ' + row[column][2:]
        elif row[column][0] == '=':
            df.loc[index,column] = 'df_input["' + column + '"] == ' + row[column][1:]
        elif row[column][0] == '*':
            if row[column][-1] == '*':
                df.loc[index,column] = 'df_input["' + column + '"].str.contains("' + row[column][1:-1] +'")'
            else:
                df.loc[index,column] = 'df_input["' + column + '"].str.startswith("' + row[column][1:] +'")'
        elif row[column][-1] == '*':
                df.loc[index,column] = 'df_input["' + column + '"].str.endswith("' + row[column][:-1] +'")'
        else:
            df.loc[index,column] = 'df_input["' + column + '"] == "' + row[column] +'"'

# print(df)

df_input = df_import.copy() # TODO Inplace?

# Überschneidene Spalten finden
column_input = df_input.columns.to_list()
column_inner = list(set(column_input) & set(columns_all)) # TODO Schon früher machen?

# Check ob result schon existiert
if tm_result in column_input:
    print(tm_result + ' ist bereits im Datenset vorhanden!') # TODO Müsste eigentlich hier abbrechen

# Test Daten aufbereiten
for column in column_inner:
    # Ersetzt Boolean Werte durch Strings
    df_input[column] = df_input[column].replace(booleanDictionary_Upper)

    # Ändert Datentyp in objects
    if df_input[column].dtype == bool:
        df_input[column] = df_input[column].astype(object)
    
    # Macht die Strings groß
    if not case_sensitive and df_input[column].dtype == object:
        df_input[column] = df_input[column].str.upper()

df_input[tm_result] = None

for index, row in df.iterrows():
    row_clean = row.dropna()
    
    row_clean_columns = row_clean.index.to_list()
    row_clean_columns.remove(tm_result)

    query_start = "df_input.loc[(df_input[" + tm_result + "].isna())"
    query_end = ", " + tm_result + "] = '" + str(row_clean[tm_result]) + "'" # TODO anderer Dtype als String?

    query_mid = ""
    for row_column in row_clean_columns:
        query_mid = query_mid + " & (" + row_clean[row_column] + ")"

    query_full = query_start + query_mid + query_end
    # print(query_full)
    exec(query_full) # TODO Risiko für Code Injektion?
    
print(df_input)
# TODO Probleme bei dem Vergleich von Float mit Int?


             StringTest BooleanTest  IntTest tm_result
0                 Alpha        True        3         1
1                  Beta        True        3         1
2                 Gamma        True        3         1
3              Hallo Du        True        4  Fallback
4             Hey Hallo        True        6         3
5  Hey Hallo, wie gehts        True        4  Fallback
6                     2       False        4         5
7                     2       False        4         5


/usr/local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/usr/local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/usr/local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
